In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-05 04:41:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.26MB/s    in 0.2s    

2021-12-05 04:41:41 (5.26 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://dshbigdatahw.s3.us-east-2.amazonaws.com/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
# https://dshactivity3-2021.s3.us-east-2.amazonaws.com/user_data.csv
spark.sparkContext.addFile(url)
review_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [6]:
print(review_df.count())
clean_review_df = review_df.dropna()
print(clean_review_df.count())
cleaner_review_df = clean_review_df.dropDuplicates()
print(cleaner_review_df.count())


2557288
2557005
2557005


In [7]:
cleaner_review_df.printSchema()


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:

# from pyspark.sql.types import IntegerType,BooleanType,DateType
# # Convert String to Integer Type
# df.withColumn("age",df.age.cast(IntegerType()))
# df.withColumn("age",df.age.cast('int'))
# df.withColumn("age",df.age.cast('integer'))

# # Using select
# df.select(col("age").cast('int').alias("age"))

# #Using selectExpr()
# df.selectExpr("cast(age as int) age")

# #Using with spark.sql()
# spark.sql("SELECT INT(age),BOOLEAN(isGraduated),DATE(jobStartDate) from CastExample")


In [9]:
from pyspark.sql.types import IntegerType,BooleanType,DateType
# cleaner_review_df = cleaner_review_df.withColumn("customer_id",cleaner_review_df.customer_id.cast(IntegerType()))

# cleaner_review_df = cleaner_review_df.withColumn("product_parent",cleaner_review_df.product_parent.cast(IntegerType()))
# cleaner_review_df = cleaner_review_df.withColumn("star_rating",cleaner_review_df.star_rating.cast(IntegerType()))
# cleaner_review_df = cleaner_review_df.withColumn("helpful_votes",cleaner_review_df.helpful_votes.cast(IntegerType()))
# cleaner_review_df = cleaner_review_df.withColumn("total_votes",cleaner_review_df.total_votes.cast(IntegerType()))

cleaner_review_df = cleaner_review_df.withColumn("review_date",cleaner_review_df.review_date.cast(DateType()))

In [10]:
cleaner_review_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [11]:
review_id_df = cleaner_review_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005P9J6VW5R3|   52047618|B004T34BA4|     767857880| 2013-08-22|
|R100G8CW43Z88H|   44491134|B00198KP7K|     483989260| 2014-01-11|
|R100GVZLWL8W5W|   27118518|B00021FLQK|      35303180| 2013-06-08|
|R100X0TTGJ0DJO|   11396402|B00GSPHS1U|     884654147| 2014-08-29|
|R10163CIAEURW2|   18683796|B001Q3M80U|     318115509| 2013-05-25|
|R101FKJY32QC1H|   15669933|B006IE17IG|     182631652| 2013-01-01|
|R101XR37UEXIQP|   33446506|B00008W71T|     489049465| 2004-04-06|
|R1025TDS5CIURA|   53010138|B000CSS520|     465533388| 2007-05-30|
|R1026XUCSMBLNC|   40950391|B0013AV9AG|     627907179| 2013-03-15|
|R102R94KOSEH61|   45904960|B001E8NXG0|      59904235| 2013-01-09|
|R103VFU1DTRU2F|   12435754|B00K2KQ2QO|     406215080| 2014-07-18|
|R103WHKEGIHZ5T|   20916354|B002S42A3Q|     613355079| 2013-08

In [12]:
products_df = cleaner_review_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004T34BA4|Achla Designs Hea...|
|B00198KP7K|Grill Daddy Repla...|
|B00021FLQK|Mr. Bar-B-Q Plati...|
|B00GSPHS1U|Intex Sand Filter...|
|B001Q3M80U|Jack Post CG-12Z ...|
|B006IE17IG|STC EGH520 Portab...|
|B00008W71T|Weber Slide Aside...|
|B000CSS520|Troy-Bilt Pony 6....|
|B0013AV9AG|Black & Decker CV...|
|B001E8NXG0|Achla Designs Din...|
|B00K2KQ2QO|KINGLAKE 100PCS W...|
|B002S42A3Q|Heavy Duty Replac...|
|B000CFOUEU|Rubbermaid  Horiz...|
|B001HB35JO|Songbird Essentia...|
|B00H4D2U9K|Eley Wall Mount G...|
|B0012XWTGW|Heavenly Blue Mor...|
|B00CAB5AAW|Buddha of Compass...|
|B00P5DTFGW|1/4 Lb (113 Grams...|
|B00025H2PY|Diatomaceous Eart...|
|B004REK370|Char-Broil Stainl...|
+----------+--------------------+
only showing top 20 rows



In [13]:
products_df.count()

2557005

In [14]:
customers_df = cleaner_review_df.groupby(["customer_id"]).count()
customers_df =customers_df.withColumnRenamed('count' , 'customer_count')
# df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)
# df_bostonLegible = df_boston.withColumnRenamed("zn", "Zoning")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13507163|             1|
|   42192220|             3|
|   16022693|             2|
|   17054071|             1|
|   31950090|             3|
|   23669123|             1|
|   20901680|             1|
|   51281657|             5|
|   42050941|             2|
|   47844876|             2|
|   23745825|             1|
|   51168318|             1|
|   16181580|             2|
|   50699952|            15|
|   43483731|             8|
|    2050707|             1|
|   47884795|             1|
|    4712293|             1|
|   37267422|             2|
|   11760241|             2|
+-----------+--------------+
only showing top 20 rows



In [15]:
vine_table_df = cleaner_review_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005P9J6VW5R3|          2|            5|          5|   N|
|R100G8CW43Z88H|          1|            1|          1|   N|
|R100GVZLWL8W5W|          5|            0|          0|   N|
|R100X0TTGJ0DJO|          5|            0|          0|   N|
|R10163CIAEURW2|          5|            4|          4|   N|
|R101FKJY32QC1H|          3|            0|          0|   N|
|R101XR37UEXIQP|          5|           24|         24|   N|
|R1025TDS5CIURA|          3|            4|          5|   N|
|R1026XUCSMBLNC|          5|            0|          0|   N|
|R102R94KOSEH61|          5|            1|          1|   N|
|R103VFU1DTRU2F|          5|            0|          0|   N|
|R103WHKEGIHZ5T|          5|            0|          0|   N|
| R104WQ0IJLWPD|          4|            5|          7|   N|
|R104XSCHM7NF6C|          2|            

Postgres Setup

In [ ]:



#  Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cl5tdbfnev78.us-east-2.rds.amazonaws.com:5432/HW"
config = {"user":"root", 
          "password": "xxrdexx", 
          "driver":"org.postgresql.Driver"}


In [17]:
# Write DataFrame to review_id_table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
# Write dataframe to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)